The Command class is used to control the flow of a running graph — especially to:

- resume execution after an interrupt,

- update or modify graph state, or

- manually influence graph behavior from outside.

1. Resume after interrupt

When a node pauses with interrupt(payload),

you use Command(resume= value / str / dict)  o continue the workflow. like

graph.invoke(Command(resume={"edited_text": "New draft"}), config=config)

In [2]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict
from langgraph.types import Command

class State(TypedDict):
    text: str

def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_c", 
        update={
            "text": state["text"] + "a"
        }
    )

def node_b(state: State): 
    print("Node B")
    return Command(
        goto=END, 
        update={
            "text": state["text"] + "b"
        }
    )


def node_c(state: State): 
    print("Node C")
    return Command(
        goto="node_b", 
        update={
            "text": state["text"] + "c"
        }
    )

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)

graph.set_entry_point("node_a")


app = graph.compile()

response = app.invoke({
    "text": ""
})

response

Node A
Node C
Node B


{'text': 'acb'}

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver


memory = MemorySaver()

class State(TypedDict):
    value: str

def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_b", 
        update={
            "value": state["value"] + "a"
        }
    )
def node_b(state: State): 
    print("Node B")

    human_response = interrupt("Do you want to go to C or D? Type C/D")

    print("Human Review Values: ", human_response)
    
    if(human_response == "C"): 
        return Command(
            goto="node_c", 
            update={
                "value": state["value"] + "b"
            }
        ) 
    elif(human_response == "D"): 
        return Command(
            goto="node_d", 
            update={
                "value": state["value"] + "b"
            }
        )


def node_c(state: State): 
    print("Node C")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "c"
        }
    )

def node_d(state: State): 
    print("Node D")
    return Command(
        goto=END, 
        update={
            "value": state["value"] + "d"
        }
)

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a") 

app = graph.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

initialState = {
    "value": ""
}

first_result = app.invoke(initialState, config, stream_mode="updates")
first_result

Node A
command value is : Command(update={'value': 'a'}, goto='node_b')
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', id='2d1f84878065acc73efc5b38a18c369c'),)}]

In [4]:
print(app.get_state(config).next)

('node_b',)


In [5]:
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

Node B
Human Review Values:  C
Node C


[{'node_b': {'value': 'ab'}}, {'node_c': {'value': 'abc'}}]